In [111]:
import numpy as np
from math import exp, sqrt
import matplotlib.pyplot as plt
import BayesODE.Bayesian as bo
from BayesODE.Tests.root_gen import root_gen
from BayesODE.utils.utils import mvCond
from BayesODE.Kalman.kalman_initial_draw import kalman_initial_draw
from BayesODE.Kalman.kalman_ode_higher import kalman_ode_higher
from BayesODE.Kalman.higher_mvCond import higher_mvCond
from BayesODE.Tests.test_exp_integrate import cov_yy_ex
from BayesODE.Kalman.cov_car import cov_car
from BayesODE.Kalman.Old.kalman_filter import kalman_filter
from BayesODE.Kalman.Old.kalman_smooth import kalman_smooth
from BayesODE.Kalman.pykalman.filter_update_full import filter_update_full
from pykalman import standard as pks
import scipy as sc
from BayesODE.Kalman.KalmanTV import KalmanTV

In [112]:
def f(x,t):
    return [x[0]+2*x[1]+t, 3*x[0]+2*x[1]]
x0 = [0,4]
v0 = f(x0, 0)

In [113]:
def f2(x,t):
    return [x[0]+2*x[2], 3*x[0]+2*x[2]]

In [114]:
def ss2gss(AA, bb, CC, DD, ee, FF):
    p = bb.shape[0]
    len_y = bb.shape[0] + ee.shape[0]
    N = ee.shape[1] - 1
    
    Aa = np.zeros((len_y, len_y, N+1))
    print(Aa.shape)
    bB = np.zeros((len_y, N+1))
    Cc = np.zeros((len_y, len_y, N+1))
    for i in range(N+1):
        #Aa
        Aan = np.zeros((len_y, len_y))
        Aan[0:p,0:p] = AA[:,:,i]
        Aan[p:,0:p] = DD[:,:,i].dot(AA[:,:,i])
        Aa[:,:,i] = Aan
        
        #bB
        bBn = np.zeros(len_y)
        bBn[0:p] = bb[:,i]
        bBn[p:] = DD[:,:,i].dot(bb[:,i]) + ee[:,i]
        bB[:,i]  = bBn
        
        #Cc
        Ccn = np.zeros((len_y, len_y))
        Ccn[0:p,0:p] = CC[:,:,i]
        Ccn[p:,0:p] = DD[:,:,i].dot(CC[:,:,i])
        Ccn[p:,p:] = FF[i]
        Cc[:,:,i] = Ccn
        
    Aa[:,:,0] = np.nan
    return Aa, bB, Cc


In [169]:
def kalman_ode_higher(fun, x_0, N, wgtState, muState, varState, wgtMeas, T=1):
   
    n_dim_meas = wgtMeas.shape[0]
    n_dim_state = len(muState)
    n_timesteps = N+1

    # allocate memory for observations
    xMeass = np.zeros((n_timesteps,n_dim_meas))

    # argumgents for kalman_filter and kalman_smooth
    muMeas = np.zeros(n_dim_meas)
    varMeass = np.zeros((n_timesteps, n_dim_meas, n_dim_meas))
    muState_filts = np.zeros((n_timesteps, n_dim_state))
    varState_filts = np.zeros((n_timesteps, n_dim_state, n_dim_state))
    muState_preds = np.zeros((n_timesteps, n_dim_state))
    varState_preds = np.zeros((n_timesteps, n_dim_state, n_dim_state))
    muState_smooths = np.zeros((n_timesteps, n_dim_state))
    varState_smooths = np.zeros((n_timesteps, n_dim_state, n_dim_state))
    xStates = np.zeros((n_timesteps, n_dim_state))

    # initialize things
    muState_filts[0] = x_0
    xMeass[0] = x_0.dot(wgtMeas.T)
    muState_preds[0] = muState_filts[0]
    varState_preds[0] = varState_filts[0]

    # forward pass
    # calculate mu_tt = E[X_t | y_0:t-1] and
    # Sigma_tt = var(X_t | y_0:t-1)

    KFS = KalmanTV(n_dim_meas, n_dim_state)
    for t in range(N):
        mu_tt = np.dot(wgtState, muState_filts[t]) + muState
        Sigma_tt = np.linalg.multi_dot([wgtState, varState_filts[t], wgtState.T]) + varState #A*Sigma[t]*A.T + V 
        varMeass[t+1] = np.linalg.multi_dot([wgtMeas, Sigma_tt, wgtMeas.T]) # new observation_covariance
        I_tt = np.random.multivariate_normal(np.zeros(n_dim_state), np.eye(n_dim_state))
        D_tt = np.linalg.cholesky(Sigma_tt)
        xState_t1 = mu_tt + D_tt.dot(I_tt) #X_{t+1} ~ N(mu_{t+1|t}, Sigma_{t+1|t})
        xMeass[t+1] = fun(xState_t1, (t+1)/N) #new observation (y_{t+1})
        (muState_preds[t+1], varState_preds[t+1], muState_filts[t+1], varState_filts[t+1]) = (
            KFS.filter(muState_past = muState_filts[t],
                    varState_past = varState_filts[t],
                    muState = muState,
                    wgtState = wgtState,
                    varState = varState,
                    xMeas = xMeass[t+1],
                    muMeas = muMeas,
                    wgtMeas = wgtMeas,
                    varMeas = varMeass[t+1])
            
        )
        if t<2:
            print("mu: {}".format(mu_tt))
            print("xMeas: {}".format(xMeass[t+1]))
            print(muState_filts[t+1])
    # backward pass
    muState_smooths[-1] = muState_filts[-1]
    varState_smooths[-1] = varState_filts[-1]
    xStates[-1] = np.random.multivariate_normal(muState_smooths[-1], varState_smooths[-1])
    for t in reversed(range(N)):
        (muState_smooths[t], varState_smooths[t], xStates[t]) = (
            KFS.smooth(xState_next = xStates[t+1],
                    muState_next = muState_smooths[t+1],
                    varState_next = varState_smooths[t+1],
                    muState_filt = muState_filts[t],
                    varState_filt = varState_filts[t],
                    muState_pred = muState_preds[t+1],
                    varState_pred = varState_preds[t+1],
                    wgtState = wgtState)
        )
    
    return xStates, muState_smooths, varState_smooths, varState_preds, xMeass, muState_filts, varState_filts


In [133]:
def mvGSS(AA, bb, CC):
    d,veclen = bb.shape
    N = veclen-1
    D = d*N + d
    An_m = np.zeros((d,d,N+1,N+1))
    for n in range(N+1):
        for m in range(N+1):
            if m>n:
                An_m[:,:,n,m] = np.eye(d)
            elif n==m:
                An_m[:,:,n,m] = AA[:,:,n]
            else:
                diff = n-m
                A_diff = AA[:,:,m]
                for i in range(diff):
                    A_diff = np.matmul(AA[:,:,m+i+1],A_diff)
                An_m[:,:,n,m] = A_diff
    L = np.zeros((D,D))
    mean_Y = np.zeros(D)
    for n in range(N+1):
        for m in range(n,N+1):
            if n == N:
                L[m*d:m*d+d,n*d:n*d+d] = np.matmul(np.eye(d), CC[:,:,n])
            else:
                L[m*d:m*d+d,n*d:n*d+d] = np.matmul(An_m[:,:,m,n+1], CC[:,:,n])
        for l in range(n):
            mean_Y[n*d:n*d+d] = mean_Y[n*d:n*d+d] + An_m[:,:,n,l+1].dot(bb[:,l])
    LL = L.dot(L.T)
    var_Y = LL
    return An_m, L, mean_Y, var_Y

In [134]:
#Define parameters
N = 2
q = 2
p = q+2

delta_t = np.array([1/N])
r0 = 0.5
sigma = 0.001
roots = root_gen(r0, p) #Generate roots to draw x^{(3)}_0
a = np.array([[0,1,0,0],[0,0,0,1]])

Y0 = np.array([x0[0],v0[0],x0[1],v0[1]])
A, V = higher_mvCond(delta_t, roots, sigma) 
lam = np.zeros(p)
b = lam - A.dot(lam)
fun = f2

In [170]:
Yn, Yn_mean, Yn_var, Sigma_preds, xMeass, muState_filts, varState_filts = kalman_ode_higher(f2, Y0, N, A, b, V, a)

mu: [  4.12860411   7.45110836  -7.07489828 -15.88759291]
xMeas: [-10.02118198  -1.7639727 ]
[  2.60815878  -1.28503681 -33.5882823   -8.8257828 ]
mu: [ -1.10622694 -10.46472524  -0.32607766  55.19859056]
xMeas: [-1.75828767 -3.97073585]
[ 0.43905835 -6.11150646  3.37610514 25.61392736]


In [137]:
C = np.linalg.cholesky(V)
D = a

AA = np.dstack([A]*(N+1))
bb = np.repeat(b[:,np.newaxis],N+1,axis=1)
CC = np.dstack([C]*(N+1))
DD = np.dstack([D]*(N+1))
ee = np.zeros((2, N+1))
FF = np.zeros((N+1, 2,2))

AA[:,:,0] = np.nan
bb[:,0] = Y0
V_inf = cov_car([], roots, sigma, v_infinity=True) 
#CC[:,:,0] = np.linalg.cholesky(V_inf)
CC[:,:,0] = np.zeros((CC.shape[0], CC.shape[0]))

for i in range(1,N+1):
    FF[i] = np.linalg.cholesky(np.linalg.multi_dot([D, Sigma_preds[i], D.T]))


In [138]:
DD[:,:,i].dot(AA[:,:,i])

array([[-0.06852673,  0.77006529,  0.26983804,  0.02640423],
       [-0.23091345, -1.00352665, -1.57120889, -0.19681802]])

In [139]:
DD[:,:,i].dot(bb[:,i])

array([0., 0.])

In [140]:
Aa, bB, Cc = ss2gss(AA,bb,CC,DD,ee,FF)

(6, 6, 3)


In [141]:
An_m, L, mean_Y, var_Y = mvGSS(Aa,bB,Cc)

In [142]:
#Check Y_1
var_YY_1 = np.linalg.multi_dot([Aa[:,:,1], var_YY_0, Aa[:,:,1].T]) + np.matmul(Cc[:,:,1],Cc[:,:,1].T)
np.allclose(var_Y[6:12, 6:12],var_YY_1)

True

In [147]:
#Kalman Filter p(Z_2|X_{0:2})
icond = np.array([False, False, False, False, True, True]*2 + [False, False, False, False, False, False]*(N-2))
R,s,T = mvCond(mean_Y[6:], var_Y[6:,6:], icond)

In [167]:
np.allclose(R.dot(xMeass[1:3].flatten())[4:] + s[4:], muState_filts[2])

True

In [171]:
T[4:8,4:8]

array([[ 3.46324872e-11,  5.66090866e-11, -5.58033775e-11,
        -2.63090137e-10],
       [ 5.66090866e-11,  1.58292612e-10,  1.42397880e-10,
        -8.81490218e-10],
       [-5.58033775e-11,  1.42397880e-10,  1.48544275e-09,
         1.74899739e-10],
       [-2.63090137e-10, -8.81490218e-10,  1.74899739e-10,
         2.90825532e-08]])

In [173]:
varState_filts[2]

array([[ 3.46324872e-11,  5.66090866e-11, -5.58033775e-11,
        -2.63090137e-10],
       [ 5.66090866e-11,  1.58292612e-10,  1.42397880e-10,
        -8.81490218e-10],
       [-5.58033775e-11,  1.42397880e-10,  1.48544275e-09,
         1.74899739e-10],
       [-2.63090137e-10, -8.81490218e-10,  1.74899739e-10,
         2.90825532e-08]])